<h1><center>Векторные представления слов</center></h1>



![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

Есть три сценария работы с векторными представлениями:
- Взять предобученную модель
- Обучить свою модель 
- Взять предобученную модель и дообучить ее


## Предобученные модели: RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятора семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный текстовый файл 
* .bin.gz — бинарный файл

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же векторы обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных моделей *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ : https://rusvectores.org/ru/models/

In [5]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/aksenov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
! ls

2_embeddings.ipynb           ru_analogy_tagged.txt
alice.txt                    ruscorpora_mean_hs.model.bin


In [8]:
# model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin'
model_path = 'ruscorpora_mean_hs.model.bin'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [9]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **Важно!** В названиях моделей на `rusvectores` указано, какой набор тегов они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [10]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print(f'Увы, слова "{word}" нет в модели!')

день_S
неделя_S :  0.7165195941925049
месяц_S :  0.631048858165741
вечер_S :  0.5828739404678345
утро_S :  0.5676207542419434
час_S :  0.5605547428131104
минута_S :  0.5297019481658936
гекатомбеон_S :  0.4897990822792053
денек_S :  0.48224714398384094
полчаса_S :  0.48217129707336426
ночь_S :  0.478074848651886


ночь_S
вечер_S :  0.6946247816085815
утро_S :  0.57301926612854
ноченька_S :  0.5582467317581177
рассвет_S :  0.5553582906723022
ночка_S :  0.5351512432098389
полдень_S :  0.5334426164627075
полночь_S :  0.478694349527359
день_S :  0.4780748784542084
сумерки_S :  0.4390218257904053
фундерфун_S :  0.4340824782848358


человек_S
женщина_S :  0.5979775190353394
парень_S :  0.4991787374019623
мужчина_S :  0.4767409563064575
мужик_S :  0.47384002804756165
россиянин_S :  0.47190436720848083
народ_S :  0.4654741883277893
согражданин_S :  0.45378512144088745
горожанин_S :  0.44368088245391846
девушка_S :  0.44314485788345337
иностранец_S :  0.43849867582321167


семантика_S
семантичес

Находим косинусную близость пары слов:

In [11]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.23895611


Построим аналогии: 
* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [12]:
print(model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


In [13]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

/Users/aksenov/opt/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'ананас_S'

## Предобученные модели: GloVe

Скачаем модель с сайта: https://nlp.stanford.edu/projects/glove/

In [14]:
! ls

2_embeddings.ipynb           ru_analogy_tagged.txt
alice.txt                    ruscorpora_mean_hs.model.bin


In [15]:
! less ./glove.6B/glove.6B.100d.txt

./glove.6B/glove.6B.100d.txt: No such file or directory


In [41]:
with open('./glove.6B.100d.txt') as f:
    line = f.readline()
    

In [47]:
import numpy as np

glove = {}
with open('./glove.6B.100d.txt', 'r') as f:
    for line in f:
        try:
            word, embedding = line.split(' ',1)
            wordEmbedding = np.array([float(value) for value in embedding[1:].split(' ')])
            glove[word] = wordEmbedding
        except:
            print(word)

print("Vocab size: ", len(glove))

hochtief
c&o
highline
Vocab size:  399997


In [48]:
len(glove['hello'])

100

In [49]:
glove['hello']

array([ 0.26688  ,  0.39632  ,  0.6169   , -0.77451  , -0.1039   ,
        0.26697  ,  0.2788   ,  0.30992  ,  0.0054685, -0.085256 ,
        0.73602  , -0.098432 ,  0.5479   , -0.030305 ,  0.33479  ,
        0.14094  , -0.0070003,  0.32569  ,  0.22902  ,  0.46557  ,
       -0.19531  ,  0.37491  , -0.7139   , -0.51775  ,  0.77039  ,
        1.0881   , -0.66011  , -0.16234  ,  0.9119   ,  0.21046  ,
        0.047494 ,  1.0019   ,  1.1133   ,  0.70094  , -0.08696  ,
        0.47571  ,  0.1636   , -0.44469  ,  0.4469   , -0.93817  ,
        0.013101 ,  0.085964 , -0.67456  ,  0.49662  , -0.037827 ,
       -0.11038  , -0.28612  ,  0.074606 , -0.31527  , -0.093774 ,
       -0.57069  ,  0.66865  ,  0.45307  , -0.34154  , -0.7166   ,
       -0.75273  ,  0.075212 ,  0.57903  , -0.1191   , -0.11379  ,
       -0.10026  ,  0.71341  , -1.1574   , -0.74026  ,  0.40452  ,
        0.18023  ,  0.21449  ,  0.37638  ,  0.11239  , -0.53639  ,
       -0.025092 ,  0.31886  , -0.25013  , -0.63283  , -0.0118

## Предобученные модели: fastText

FastText использует не только векторы слов, но и векторы n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Руководство](https://fasttext.cc/docs/en/support.html)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

На сайте проекта можно найти предобученные модели для 157 языков (в том числе русского): https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
import fasttext

#fasttext.util.download_model('ru', if_exists='ignore')
ft = fasttext.load_model('cc.ru.300.bin')

In [1]:
!ls

2_embeddings.ipynb           glove.6B.100d.txt
alice.txt                    ru_analogy_tagged.txt
cc.en.300.bin                ruscorpora_mean_hs.model.bin
cc.en.300.bin.gz


In [ ]:
ft['модель']

У fasttext есть все те же методы, что в gensim, но называются они иначе:

In [ ]:
ft.get_nearest_neighbors('чай')

In [ ]:
ft.get_analogies("женщина", "мужчина", "актер")

Важная особенность: так как модель обучена на символьных n-граммах, нет проблемы OOV (out of vocabulary) слов:

In [ ]:
ft.get_nearest_neighbors('книжонка')

In [ ]:
ft.get_nearest_neighbors('компютер')

## Как обучить свою модель: word2vec

В качестве обучающих данных возьмем англоязычные отзывы о фильмах с сайта IMDB (данные взяты отсюда http://ai.stanford.edu/~amaas/data/sentiment/).

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [3]:
data = pd.read_csv("IMDB Dataset.csv")

len(data)

FileNotFoundError: [Errno 2] File IMDB Dataset.csv does not exist: 'IMDB Dataset.csv'

In [ ]:
data.head()

Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приведем все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [ ]:
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer=tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [ ]:
with Pool(4) as p:
    sentences = list(tqdm(p.imap(review_to_sentences, data["review"]), total=len(data)))

print(len(sentences))
print(*sentences[:3])

In [ ]:
flat_sentences = [item for sublist in sentences for item in sublist]
len(flat_sentences)

In [ ]:
# это понадобится нам позже для обучения модели fasttext

with open('clean_text.txt', 'w') as f:
    for s in flat_sentences:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**Важно!** Обратите внимание, что тренировка модели не включает предобработку! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [ ]:
print("Training model...")

%time model_en = word2vec.Word2Vec(flat_sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Смотрим, сколько в модели слов.

In [ ]:
print(len(model_en.wv.vocab))

Попробуем оценить модель вручную, аналогично тому, как проверяли предобученные модели.

In [ ]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))

In [ ]:
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

In [ ]:
print(model_en.wv.most_similar("usa", topn=3))

In [ ]:
print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

### Почему важно, на каких данных обучалась модель?

Посмотрим на ближайшие по смыслу слова к слову "star":
- в модели, обученной на обзорах фильмов
- в модели, обученной на Википедии

In [ ]:
print(*model_en.wv.most_similar("star", topn=10), sep='\n')

In [ ]:
model_en.similarity('star', 'celebrity')

In [ ]:
model_en.similarity('star', 'sky')

In [ ]:
model_en.similarity('star', 'shine')

Скачаем предобученную модель fastText для английского:

In [18]:
import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore') 

ft_eng = fasttext.load_model('cc.en.300.bin')

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
cosine_similarity([ft_eng['star']], [ft_eng['celebrity']])

array([[0.48565802]], dtype=float32)

In [20]:
cosine_similarity([ft_eng['star']], [ft_eng['sky']])

array([[0.2991682]], dtype=float32)

In [21]:
cosine_similarity([ft_eng['star']], [ft_eng['shine']])

array([[0.2885157]], dtype=float32)

### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [22]:
model_en.wv.similarity('white', 'rabbit')

NameError: name 'model_en' is not defined

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [ ]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [ ]:
model_path = "movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

In [ ]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

"Белый" и "кролик" стали ближе друг к другу!

In [ ]:
model.wv.similarity('white', 'rabbit')

## Как обучить свою модель: fastText

Выше мы записали предобработанные данные IMDB в текстовый файл, теперь мы можем использовать его для обучения модели fastText.

In [ ]:
import fasttext

# так можно обучить свою модель 
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

In [ ]:
ft_model.get_word_vector("movie")

In [ ]:
ft_model.get_nearest_neighbors('actor')

In [ ]:
ft_model.get_analogies("woman", "man", "actor")

OOV:

In [ ]:
ft_model.get_nearest_neighbors('actr')

In [ ]:
ft_model.get_nearest_neighbors('moviegeek')

## Оценка

Мы научились обучать модели, научились загружать готовые, а как понять, какая модель лучше? Или вот, например, мы обучили модель, а как понять, насколько она хорошая? Рассмотрим два метода: с помощью метрик, основанных на подготовленных данных, и визуализации.

Для этого существуют специальные наборы данных для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Сибирь и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного набора данных мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Набор данных будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Данные для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [ ]:
res = model_ru.accuracy('ru_analogy_tagged.txt')

In [ ]:
!less ru_analogy_tagged.txt

In [ ]:
print(res[4]['incorrect'][:10])

In [ ]:
print(model_ru.most_similar(positive=['мальчик_S', 'бабка_S'], negative=['девочка_S'])[0][0])

In [ ]:
len(res[4]['incorrect']), len(res[4]['correct'])

## Визуализация

На полученную модель можно посмотреть, визуализировав ее, например, на плоскости.
### t-SNE

**t-SNE**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).


Изобразим на плоскости 1000 самых частотных слов из коллекции текстов про кино:

In [ ]:
model_path = "movie_reviews.model"

model = word2vec.Word2Vec.load(model_path)

In [ ]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(flat_sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])
    
print(top_words[:50:])
top_words_vec = model[top_words]

In [ ]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Чтобы вычислить преобразование t-SNE быстрее (и иногда еще и эффективнее), можно сперва снизить размерность исходных данных с помощью, например, SVD, и потом применять t-SNE:

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_50 = TruncatedSVD(n_components=50)
top_words_vec_50 = svd_50.fit_transform(top_words_vec)
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

In [ ]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)